# Prepare Habex telescope and coronagraph

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import hcipy as hc

## Parameters

In [ ]:
nPup = 1000
D_pup = 4.
samp_foc = 4
rad_foc = 36
wavelength = 500e-9
pupil_grid = hc.make_pupil_grid(nPup, D_pup)
telescope_pupil = hc.aperture.circular_aperture(D_pup)
telescope_pupil_field = hc.evaluate_supersampled(telescope_pupil, pupil_grid, 1)
lyot_stop = hc.aperture.circular_aperture(D_pup * 0.95)
lyot_stop_field = hc.evaluate_supersampled(lyot_stop,pupil_grid, 1)

In [ ]:
hc.imshow_field(telescope_pupil_field)
plt.title('Primary mirror')

In [ ]:
hc.imshow_field(lyot_stop_field)
plt.title('Lyot stop')

In [ ]:
focal_grid = hc.make_focal_grid(
    samp_foc,
    rad_foc,
    pupil_diameter= D_pup,
    focal_length=1.,
    reference_wavelength= wavelength,
)

## Propagators

In [ ]:
?hc.VortexCoronagraph

In [ ]:
prop = hc.FraunhoferPropagator(pupil_grid, focal_grid)
charge = 6
# coro = hc.VortexCoronagraph(pupil_grid, charge,scaling_factor = 4,q = 2048,window_size = 128)
coro = hc.VortexCoronagraph(pupil_grid, charge)
lyot_stop = hc.Apodizer(lyot_stop_field)

## Actual code for propagation

In [ ]:
wf = hc.Wavefront(telescope_pupil_field, wavelength)
lyot_plane = coro.forward(wf)
post_lyot_mask = lyot_stop(lyot_plane)
post_lyot_mask.wavelength = wavelength
img = prop(post_lyot_mask).intensity
img_ref = prop(wf).intensity

In [ ]:
plt.figure(figsize=(10, 10))
hc.imshow_field(np.log10(img_ref / img_ref.max()), vmin=-7, cmap='inferno')
plt.colorbar()
plt.show()

plt.figure(figsize=(10, 10))
hc.imshow_field(np.log10(img / img_ref.max()), cmap='inferno', vmin=-12, vmax=-5)
plt.colorbar()
plt.show()